In [1]:
pip install pymysql pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.5 MB/s eta 0:00:00


In [2]:
# Step 1: Install MySQL Server
!apt-get update
!apt-get install -y mysql-server

# Step 2: Start the MySQL service
!service mysql start

# Step 3: Set up a sample MySQL database and user
!mysql -e "CREATE DATABASE colab_db;"  # Create a new database named 'colab_db'
!mysql -e "CREATE USER 'colab_user'@'localhost' IDENTIFIED BY 'password';"  # Create a new user
!mysql -e "GRANT ALL PRIVILEGES ON colab_db.* TO 'colab_user'@'localhost';"  # Grant all privileges to the user
!mysql -e "FLUSH PRIVILEGES;"  # Refresh privileges


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,552 kB]
Get:13 http://archiv

In [4]:
# Step 4: Python code to connect to the MySQL server
import pymysql

try:
    # Connect to the MySQL server running in Colab
    connection = pymysql.connect(
        host='127.0.0.1',
        port=3306,
        user='colab_user',  # User created earlier
        password='password',  # Password created earlier
        database='colab_db'  # Database created earlier
    )
    print("Connected to the database!")

    # Step 5: Create a cursor to execute queries
    cursor = connection.cursor()

    # Example: Create a table
    cursor.execute("CREATE TABLE IF NOT EXISTS users (id INT, name VARCHAR(255))")
    print("Table created or already exists.")

    # Example: Insert data into the table
    cursor.execute("INSERT INTO users (id, name) VALUES (1, 'John Doe')")
    connection.commit()
    print("Data inserted!")

    # Example: Query data from the table
    cursor.execute("SELECT * FROM users")
    results = cursor.fetchall()
    print("Query Results:", results)

except pymysql.MySQLError as e:
    print("Error:", e)

finally:
    if connection:
        connection.close()
        print("Connection closed.")

Connected to the database!
Table created or already exists.
Data inserted!
Query Results: ((1, 'John Doe'), (1, 'John Doe'))
Connection closed.


# Hands On Activity! SQL in Python!!

### **Exercise 1: Create Tables**

1. Create a table named `products` with the following columns:

   * `product_id`: INT (Primary Key)
   * `product_name`: VARCHAR(255)
   * `price`: DECIMAL(10, 2)

2. Create a table named `categories` with the following columns:

   * `category_id`: INT (Primary Key)
   * `category_name`: VARCHAR(255)

3. Create a table named `product_categories` to represent the many-to-many relationship between `products` and `categories`:

   * `product_id`: INT (Foreign Key)
   * `category_id`: INT (Foreign Key)

---

### **Exercise 2: Insert Data**

1. Insert at least 5 products into the `products` table with various prices.
2. Insert at least 3 categories into the `categories` table.
3. Associate each product with one or more categories in the `product_categories` table.

---

### **Exercise 3: Write Queries**

1. Retrieve all products with their prices.
2. Retrieve all categories and their associated products.
3. Find the most expensive product in each category.
4. Calculate the total number of products in each category.
5. Find categories that don’t have any products.

---

### **Exercise 4: Modify Data**

1. Update the price of a specific product.
2. Delete a product and ensure its relationships in `product_categories` are also removed.

---

### **Exercise 5: Advanced Queries**

1. Write a query to find the top 3 most expensive products.
2. Retrieve products with prices above the average product price.
3. Create a stored procedure to add a new product along with its category associations.


In [6]:
# EX : 1

import sqlite3

# Connect to SQLite database (in-memory for testing)
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Enable foreign key constraint enforcement
cursor.execute("PRAGMA foreign_keys = ON;")

# Create products table
cursor.execute('''
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name VARCHAR(255),
    price DECIMAL(10, 2)
)
''')

# Create categories table
cursor.execute('''
CREATE TABLE categories (
    category_id INTEGER PRIMARY KEY,
    category_name VARCHAR(255)
)
''')

# Create product_categories table for many-to-many relationship
cursor.execute('''
CREATE TABLE product_categories (
    product_id INTEGER,
    category_id INTEGER,
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
)
''')

print("Tables created successfully.")


Tables created successfully.


In [7]:
# EX : 2

# Insert data into products table
products_data = [
    (1, 'Laptop', 1200.00),
    (2, 'Smartphone', 800.00),
    (3, 'Keyboard', 45.99),
    (4, 'Monitor', 200.00),
    (5, 'Headphones', 75.50)
]
cursor.executemany('INSERT INTO products VALUES (?, ?, ?)', products_data)

# Insert data into categories table
categories_data = [
    (1, 'Electronics'),
    (2, 'Accessories'),
    (3, 'Office Equipment')
]
cursor.executemany('INSERT INTO categories VALUES (?, ?)', categories_data)

# Associate products with categories in product_categories table
product_categories_data = [
    (1, 1),  # Laptop - Electronics
    (2, 1),  # Smartphone - Electronics
    (3, 2),  # Keyboard - Accessories
    (4, 1),  # Monitor - Electronics
    (4, 3),  # Monitor - Office Equipment
    (5, 2)   # Headphones - Accessories
]
cursor.executemany('INSERT INTO product_categories VALUES (?, ?)', product_categories_data)

# Commit changes to the database
conn.commit()

print("Data inserted successfully.")


Data inserted successfully.


In [8]:
print("\nProducts:")
for row in cursor.execute('SELECT * FROM products'):
    print(row)

print("\nCategories:")
for row in cursor.execute('SELECT * FROM categories'):
    print(row)

print("\nProduct-Categories:")
for row in cursor.execute('SELECT * FROM product_categories'):
    print(row)



Products:
(1, 'Laptop', 1200)
(2, 'Smartphone', 800)
(3, 'Keyboard', 45.99)
(4, 'Monitor', 200)
(5, 'Headphones', 75.5)

Categories:
(1, 'Electronics')
(2, 'Accessories')
(3, 'Office Equipment')

Product-Categories:
(1, 1)
(2, 1)
(3, 2)
(4, 1)
(4, 3)
(5, 2)


In [9]:
# EX : 3
# Retrieve all products with their prices.

print("All products with prices:")
cursor.execute('''
SELECT product_name, price
FROM products
''')
for row in cursor.fetchall():
    print(row)


All products with prices:
('Laptop', 1200)
('Smartphone', 800)
('Keyboard', 45.99)
('Monitor', 200)
('Headphones', 75.5)


In [10]:
# Retrieve all categories and their associated products.

print("\nCategories and their associated products:")
cursor.execute('''
SELECT c.category_name, p.product_name
FROM categories c
JOIN product_categories pc ON c.category_id = pc.category_id
JOIN products p ON pc.product_id = p.product_id
''')
for row in cursor.fetchall():
    print(row)



Categories and their associated products:
('Electronics', 'Laptop')
('Electronics', 'Smartphone')
('Accessories', 'Keyboard')
('Electronics', 'Monitor')
('Office Equipment', 'Monitor')
('Accessories', 'Headphones')


In [11]:
# Find the most expensive product in each category.

print("\nMost expensive product in each category:")
cursor.execute('''
SELECT c.category_name, p.product_name, MAX(p.price) as max_price
FROM categories c
JOIN product_categories pc ON c.category_id = pc.category_id
JOIN products p ON pc.product_id = p.product_id
GROUP BY c.category_id
''')
for row in cursor.fetchall():
    print(row)



Most expensive product in each category:
('Electronics', 'Laptop', 1200)
('Accessories', 'Headphones', 75.5)
('Office Equipment', 'Monitor', 200)


In [12]:
#Calculate the total number of products in each category

print("\nTotal number of products in each category:")
cursor.execute('''
SELECT c.category_name, COUNT(DISTINCT pc.product_id) as total_products
FROM categories c
LEFT JOIN product_categories pc ON c.category_id = pc.category_id
GROUP BY c.category_id
''')
for row in cursor.fetchall():
    print(row)



Total number of products in each category:
('Electronics', 3)
('Accessories', 2)
('Office Equipment', 1)


In [13]:
#Find categories that don’t have any products

print("\nCategories with no associated products:")
cursor.execute('''
SELECT c.category_name
FROM categories c
LEFT JOIN product_categories pc ON c.category_id = pc.category_id
WHERE pc.product_id IS NULL
''')
for row in cursor.fetchall():
    print(row)



Categories with no associated products:


In [14]:
# EX : 4
# Update the price of a specific product.

cursor.execute('''
UPDATE products
SET price = 1100.00
WHERE product_name = 'Laptop'
''')
conn.commit()

print("Updated price of Laptop.")


Updated price of Laptop.


In [15]:
#Delete a product and ensure its relationships in product_categories are also removed.

# Get the product_id of 'Keyboard'
cursor.execute("SELECT product_id FROM products WHERE product_name = 'Keyboard'")
keyboard_id = cursor.fetchone()

if keyboard_id:
    keyboard_id = keyboard_id[0]

    # Delete related entries in product_categories
    cursor.execute('DELETE FROM product_categories WHERE product_id = ?', (keyboard_id,))

    # Delete the product itself
    cursor.execute('DELETE FROM products WHERE product_id = ?', (keyboard_id,))

    conn.commit()
    print("Deleted 'Keyboard' and its associations.")
else:
    print("'Keyboard' not found.")


Deleted 'Keyboard' and its associations.


In [16]:
# EX : 5
#Write a query to find the top 3 most expensive products.

print("Top 3 most expensive products:")
cursor.execute('''
SELECT product_name, price
FROM products
ORDER BY price DESC
LIMIT 3
''')
for row in cursor.fetchall():
    print(row)


Top 3 most expensive products:
('Laptop', 1100)
('Smartphone', 800)
('Monitor', 200)


In [17]:
#Retrieve products with prices above the average product price.

print("\nProducts with prices above average:")
cursor.execute('''
SELECT product_name, price
FROM products
WHERE price > (SELECT AVG(price) FROM products)
''')
for row in cursor.fetchall():
    print(row)



Products with prices above average:
('Laptop', 1100)
('Smartphone', 800)


In [18]:
#Create a stored procedure to add a new product along with its category associations.

def add_product_with_categories(product_name, price, category_ids):
    # Insert the new product
    cursor.execute('''
        INSERT INTO products (product_name, price) VALUES (?, ?)
    ''', (product_name, price))

    # Get the new product_id
    product_id = cursor.lastrowid

    # Insert into product_categories
    for cat_id in category_ids:
        cursor.execute('''
            INSERT INTO product_categories (product_id, category_id) VALUES (?, ?)
        ''', (product_id, cat_id))

    conn.commit()
    print(f"Product '{product_name}' added with categories {category_ids}.")


In [19]:
add_product_with_categories("Tablet", 650.00, [1, 2])  # Electronics + Accessories

Product 'Tablet' added with categories [1, 2].
